In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import warnings
warnings.filterwarnings('ignore')

mpl.rcParams['font.family'] = 'NanumGothic'
mpl.rcParams['axes.unicode_minus'] = False

# 자전거 수요 예측

In [2]:
df = pd.read_csv('bike.csv')
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [3]:
# 맞춰도 의미 없는 연도와 날짜 정보는 제거
df['datetime'] = df.datetime.apply(pd.to_datetime)

# df['year'] = df.datetime.apply(lambda x : x.year)
df['month'] = df.datetime.apply(lambda x : x.month)
# df['day'] = df.datetime.apply(lambda x : x.day)
df['hour'] = df.datetime.apply(lambda x : x.hour)

In [4]:
drop_columns = ['datetime', 'casual', 'registered']
df.drop(drop_columns, axis=1, inplace=True)

In [5]:
df.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,month,hour
0,1,0,0,1,9.84,14.395,81,0.0,16,1,0
1,1,0,0,1,9.02,13.635,80,0.0,40,1,1
2,1,0,0,1,9.02,13.635,80,0.0,32,1,2
3,1,0,0,1,9.84,14.395,75,0.0,13,1,3
4,1,0,0,1,9.84,14.395,75,0.0,1,1,4


In [6]:
y_target = df['count']
X_features = df.drop(['count'], axis=1)

In [7]:
from xgboost import XGBRegressor

xgb = XGBRegressor()
xgb.fit(X_features.values, y_target.values)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [8]:
y_target.describe()

count    10886.000000
mean       191.574132
std        181.144454
min          1.000000
25%         42.000000
50%        145.000000
75%        284.000000
max        977.000000
Name: count, dtype: float64

In [9]:
from genetic_algorithm import GeneticAlgorithm

ga = GeneticAlgorithm()

In [30]:
def fitness(indivisual, model, goal):
    pred = model.predict([indivisual])[0]
    
    return abs(pred-goal)

condition = X_features[X_features['month']==1].describe().to_dict()
n_generation = 300
population = 100
best_sample = 20
lucky_few = 20
n_child = 5
chance_of_mutation = 10
fix_features = {'month':1}
base = X_features.iloc[0]

goal = 400
tolerance = 5

In [31]:
best = ga.search(
    condition=condition,
    fitness=lambda x:fitness(x, xgb, goal),
    n_generation=n_generation,
    population=population,
    best_sample=best_sample,
    lucky_few=lucky_few,
    n_child=n_child,
    chance_of_mutation=chance_of_mutation,
    fix_features=fix_features,
    tolerance=tolerance,
    base=base,
    es=None,
    verbose=True,
    time_limit=None,
)

Generation 1/300
Elapsed Time: 0.08s, Loss: 44.981964111328125
Generation 2/300
Elapsed Time: 0.06s, Loss: 21.012359619140625
SUCESSED at generation 3/300
Elapsed Time: 0.04s, Loss: 2.6785888671875


In [32]:
xgb.predict(np.array([best]))

array([397.3214], dtype=float32)

In [33]:
best

season         1.000
holiday        0.460
workingday     0.692
weather        3.126
temp          19.144
atemp         20.521
humidity      40.836
windspeed     27.157
month          1.000
hour          18.148
dtype: float64